In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import os,sys
opj = os.path.join
from copy import deepcopy
import pickle as pkl
from tqdm import tqdm

from sim_cosmology import p, load_dataloader_and_pretrained_model
p.data_path = '../../src/dsets/cosmology/data'
p.model_path = '../../src/dsets/cosmology/data'
# adaptive-wavelets modules
from losses import get_loss_f
from train import Trainer
from evaluate import Validator
from transform2d import DWT2d
from utils import get_1dfilts, get_2dfilts, get_wavefun
from wave_attributions import Attributer
from visualize import cshow, plot_1dfilts, plot_2dfilts, plot_2dreconstruct

## load data and model

In [ ]:
# get dataloader and model
(train_loader, test_loader), model = load_dataloader_and_pretrained_model(p, img_size=p.img_size[2])

# # check prediction
# with torch.no_grad():
#     result = {'y': [], 'pred': []}
#     for data, params in train_loader:
#         result['y'].append(params[:,1].detach().cpu())
#         result['pred'].append(model(data.to(device))[:,1].detach().cpu())
# plt.scatter(torch.cat(result['y']), torch.cat(result['pred']))
# plt.xlabel('true param')
# plt.ylabel('predicted param')
# plt.show()  

In [ ]:
# get image
data = iter(test_loader).next()[0].to(device)

# wavelet transform 
wt = DWT2d(wave='db5', mode='zero', J=4, init_factor=1, noise_factor=0.1).to(device)

data_t = wt(data)
recon = wt.inverse(data_t)

print("Reconstruction error={:.5f}".format(torch.norm(recon - data)**2/data.size(0)))

# get 2d wavelet filters
filt = get_2dfilts(wt)

In [ ]:
# plot original and reconstruction images
plot_2dreconstruct(data, recon)

In [ ]:
# plot wavelet filters
plot_1dfilts(filt[0], figsize=(1,1))

In [ ]:
# plot wavelet filters
plot_2dfilts(filt[1], figsize=(5,5))

## optimize filter

In [ ]:
# train
params = list(wt.parameters())
optimizer = torch.optim.Adam(params, lr=0.001)
loss_f = get_loss_f(lamlSum=1, lamhSum=1, lamL2norm=1, lamCMF=1, lamConv=1, lamL1wave=0.1, lamL1attr=0.01)
trainer = Trainer(model, wt, optimizer, loss_f, target=1, 
                  use_residuals=True, attr_methods='Saliency', device=device, n_print=1)

In [ ]:
trainer(train_loader, epochs=2)

In [ ]:
plt.plot(np.log(trainer.train_losses))
plt.xlabel("epochs")
plt.ylabel("log train loss")
plt.title('Log-train loss vs epochs')
plt.show()

In [ ]:
data_t = wt(data)
recon = wt.inverse(data_t)

print("Reconstruction error={:.5f}".format(torch.norm(recon - data)**2/data.size(0)))

# get 2d wavelet filters
filt = get_2dfilts(wt)

In [ ]:
# plot original and reconstruction images
plot_2dreconstruct(data, recon)

## test error

In [ ]:
# validator 
validator = Validator(model, test_loader)
rec_loss, lsum_loss, hsum_loss, L2norm_loss, CMF_loss, conv_loss, L1wave_loss, L1saliency_loss, L1inputxgrad_loss = validator(wt, target=1)

# original wavelet transform
wt_o = DWT2d(wave='db5', mode='zero', J=4, init_factor=1, noise_factor=0).to(device)
rec_loss_o, lsum_loss_o, hsum_loss_o, L2norm_loss_o, CMF_loss_o, conv_loss_o, L1wave_loss_o, L1saliency_loss_o, L1inputxgrad_loss_o = validator(wt_o, target=1)

print("\n\n \t Original filter:Reconstruction Error={:.5f} lsum loss={:.5f} hsum loss={:.5f} L2norm loss={:.5f} CMF loss={:.5f} conv loss={:.5f} L1wave loss={:.5f} L1saliency loss={:.5f} L1inputxgrad loss={:.5f} \n \
        Adaptive filter:Reconstruction Error={:.5f} lsum loss={:.5f} hsum loss={:.5f} L2norm loss={:.5f} CMF loss={:.5f} conv loss={:.5f} L1wave loss={:.5f} L1saliency loss={:.5f} L1inputxgrad loss={:.5f}"\
          .format(rec_loss_o, lsum_loss_o, hsum_loss_o, L2norm_loss_o, CMF_loss_o, conv_loss_o, L1wave_loss_o, L1saliency_loss_o, L1inputxgrad_loss_o, 
                  rec_loss, lsum_loss, hsum_loss, L2norm_loss, CMF_loss, conv_loss, L1wave_loss, L1saliency_loss, L1inputxgrad_loss))

In [ ]:
# plot wavelet filters
filt = get_2dfilts(wt)
plot_1dfilts(filt[0], figsize=(1,1))
plot_2dfilts(filt[1], figsize=(5,5))

In [ ]:
# plot original wavelet filters
filt_o = get_2dfilts(wt_o)
plot_1dfilts(filt_o[0], figsize=(1,1))
plot_2dfilts(filt_o[1], figsize=(5,5))

In [ ]:
# sys.path.append('../../lib/trim')
# from trim import TrimModel
# from utils import tuple_to_tensor, tensor_to_tuple

# mt = TrimModel(model, wt.inverse, use_residuals=True)    
# attributer = Attributer(mt, attr_methods='Saliency', device='cuda')

# b = torch.tensor([]).to(device)
# c = torch.tensor([]).to(device)
# for data, _ in test_loader:
#     data = data.to(device)
#     data_t = wt(data)
#     attributions = attributer(data_t, target=1, additional_forward_args=deepcopy(data))
#     y, _ = tuple_to_tensor(data_t)
#     b = torch.cat((y,b), dim=0)     
#     z, _ = tuple_to_tensor(attributions)
#     c = torch.cat((z,c), dim=0)
# b = b.reshape(-1)
# c = c.reshape(-1)
# sort_indexes = torch.argsort(b, dim=0, descending=False)          
    
# plt.plot(b[sort_indexes].detach().cpu())
# plt.show()

# plt.plot(c[sort_indexes].detach().cpu())
# plt.show()